# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
#load csv into dataframe
load_csv = "../output_data/cities.csv"
city_df = pd.read_csv(load_csv)
city_df.head()


Unnamed: 0            City  Cloudiness Country        Date  Humidity  \
0           0          atuona          33      PF  1624091360        77   
1           1           harer          92      ET  1624091360        53   
2           2       busselton         100      AU  1624091362        73   
3           3     mys shmidta          86      RU  1624091362        33   
4           4  port elizabeth          12      CD  1624091363        54   

       Lat       Lng  Max Temp  Wind Speed  
0  -9.8000 -139.0333     79.41       22.41  
1   9.3094   42.1258     70.79       11.05  
2 -33.6500  115.3333     60.04       15.90  
3  62.0589   48.6201     80.20       14.58  
4  -7.0349   29.7638     77.68       12.80

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = city_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = city_df.loc[(city_df["Wind Speed"] <= 10) & (city_df["Cloudiness"] == 0) & \
                                   (city_df["Max Temp"] >= 70) & (city_df["Max Temp"] <= 80)].dropna()

narrowed_city_df

Unnamed: 0           City  Cloudiness Country        Date  Humidity  \
100         100         amasya           0      TR  1624091435        47   
117         117          alofi           0      NU  1624091448        94   
212         212        atambua           0      ID  1624091521        71   
220         220  half moon bay           0      LR  1624091527        81   
244         244         mhango           0      TZ  1624091548        53   
328         328         lobito           0      AO  1624091614        61   
382         382  pangkalanbuun           0      ID  1624091654        97   
407         407          dakar           0      SN  1624091672        78   
433         433      belyy yar           0      RU  1624091691        46   

         Lat       Lng  Max Temp  Wind Speed  
100  40.6667   35.8333     78.48        3.42  
117 -19.0595 -169.9187     73.29        8.05  
212  -9.1061  124.8925     77.49        2.24  
220   6.3807  -10.6953     75.09        2.84  
244  -3.2833   32.8500     73.44        6.85  
328 -12.3481   13.5456     78.80        3.87  
382  -2.6833  111.6167     75.25        4.50  
407  14.6937  -17.4441     77.13        6.91  
433  55.8402   32.9419     78.96        5.17

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

City Country      Lat       Lng Hotel Name
100         amasya      TR  40.6667   35.8333           
117          alofi      NU -19.0595 -169.9187           
212        atambua      ID  -9.1061  124.8925           
220  half moon bay      LR   6.3807  -10.6953           
244         mhango      TZ  -3.2833   32.8500           
328         lobito      AO -12.3481   13.5456           
382  pangkalanbuun      ID  -2.6833  111.6167           
407          dakar      SN  14.6937  -17.4441           
433      belyy yar      RU  55.8402   32.9419

In [10]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [11]:
import time
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 100: amasya.
Closest hotel in amasya is Sarı Konak Boutique & SPA Hotel.
------------
Retrieving Results for Index 117: alofi.
Closest hotel in alofi is Taloa Heights.
------------
Retrieving Results for Index 212: atambua.
Closest hotel in atambua is Hotel King Star.
------------
Retrieving Results for Index 220: half moon bay.
Missing field/result... skipping.
------------
Retrieving Results for Index 244: mhango.
Missing field/result... skipping.
------------
Retrieving Results for Index 328: lobito.
Closest hotel in lobito is Terminus Hotel.
------------
Retrieving Results for Index 382: pangkalanbuun.
Closest hotel in pangkalanbuun is Grand Kecubung Hotel.
------------
Retrieving Results for Index 407: dakar.
Closest hotel in dakar is Radisson Blu Hotel, Dakar Sea Plaza.
------------
Retrieving Results for Index 433: belyy yar.
Closest hotel in belyy yar is Hotel "obshatsya".
------------
-------End of Search-------


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Add marker layer ontop of heat map
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

NameError: name 'hotel_info' is not defined